In [ ]:
from  train_highway_env import *
from bokeh.io import output_notebook
from bokeh.plotting import show
import rlrom.plots
output_notebook()  # Add this at the top of your notebook

In [ ]:
cfg = utils.load_cfg('cfg_main.yml')    
tester = HwEnvTester(cfg)
#tester.run_seed(seed=0, num_steps=100)
tester.run_seed(num_steps=100)


In [ ]:
OBS = tester.env.episode['observations']

In [ ]:
lay = """            
    ego_vx
    sat(safe)
    obs_safe_hor_0
    """
fig, status=  rlrom.plots.get_fig(tester.env, lay)
show(fig)